<a href="https://colab.research.google.com/github/oviyajay/ML-Final-Project/blob/main/MLfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import os 
import cv2
!pip install tflearn
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 

#Members:...
#https://blog.paperspace.com/image-captioning-with-ai/
#image preprocessing

# TEST_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/with_mask"
# TEST_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/test/without_mask"
# TRAIN_W_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/with_mask"
# TRAIN_WO_MASK="https://github.com/oviyajay/ML-Final-Project/tree/main/maskdata/maskdata/train/without_mask"

!git clone https://github.com/oviyajay/ML-Final-Project

#label the images
#create training data and test data 


fatal: destination path 'ML-Final-Project' already exists and is not an empty directory.


In [ ]:
#Eric 
train_path = "/content/ML-Final-Project/maskdata/maskdata/train/"
os.listdir(train_path)

test_path = "/content/ML-Final-Project/maskdata/maskdata/test/"
os.listdir(test_path)



['with_mask', 'without_mask']

In [53]:
label_names = ['without_mask', 'with_mask']
IMG_SIZE = 150
LR = 1e-3
MODEL_NAME = 'withMaskvsnoMask-{}-{}.model'.format(LR, '6conv-basic')

def load_data(path):
  data = []

  for label in label_names:
    for filename in os.listdir(os.path.join(path, label)):
    
      img_file = os.path.join(path, label, filename)
      
      # loading the image from the path and then converting them into 
      # greyscale for easier covnet prob 
      img = cv2.imread(img_file, cv2.IMREAD_COLOR)

      # resizing the image for processing them in the covnet 
      img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

      #withmask: id = 1, withoutmask: id = 0
      label_id = label_names.index(label)
  

      # final step-forming the training data list with numpy array of the images 
      data.append([np.array(img), np.array(label_id)]) 

  return data



In [54]:
train_data = load_data(train_path)
test_data = load_data(test_path)


[[array([[[254, 254, 254],
        [250, 254, 254],
        [ 88,  96,  95],
        ...,
        [ 47,  68,  61],
        [ 47,  64,  60],
        [ 49,  66,  63]],

       [[253, 253, 253],
        [251, 254, 254],
        [ 59,  64,  63],
        ...,
        [ 49,  74,  64],
        [ 52,  72,  63],
        [ 44,  58,  53]],

       [[252, 254, 254],
        [250, 254, 254],
        [ 65,  68,  68],
        ...,
        [ 86, 121, 106],
        [ 64,  93,  81],
        [ 54,  77,  71]],

       ...,

       [[253, 255, 254],
        [254, 254, 254],
        [255, 253, 255],
        ...,
        [177, 182, 213],
        [213, 213, 239],
        [211, 204, 227]],

       [[255, 255, 255],
        [254, 254, 254],
        [255, 254, 253],
        ...,
        [125, 137, 175],
        [152, 162, 196],
        [179, 183, 215]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [166, 171, 198],
        [131, 140, 164],
        [158, 161, 186

In [58]:
# train is a list of [x, y] pairs
train_x = [i[0] for i in train_data]  #i[0] is a numpy array that is the size/shape of a single image
train_x = np.array(train_x, dtype=float).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # num_image, width, height, channels
train_y = [i[1] for i in train_data]
train_y = np.array(train_y, dtype=float)
test_x = [i[0] for i in test_data]
test_x = np.array(test_x).reshape(-1, IMG_SIZE, IMG_SIZE, 1) 
test_y = [i[1] for i in test_data]
print(train_x.shape)
print(train_y.shape)


(1848, 150, 150, 1)
(616,)


In [ ]:
#Iant
#tf.reset_default_graph() 
cnn = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input') 

#convolution and pooling operations
#as for the arguments, (convolutional network, numbers of channels, filter size(if one number 'x' is passed, then both dimensions are 'x' ), activation function used)
#in our case, we're starting with 16 channels, gradually working up to 128. By convention, the number of channels gradually increase or stay the same as it works its way further into the CNN
#filter size changed from 5x5 to 3x3

cnn = conv_2d(cnn, 16, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 

cnn = conv_2d(cnn, 32, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = conv_2d(cnn, 64, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = conv_2d(cnn, 128, 3, activation ='relu') 
cnn = max_pool_2d(cnn, 3) 
  
cnn = fully_connected(cnn, 1024, activation ='relu') 
cnn = dropout(cnn, 0.8) 
  
cnn = fully_connected(cnn, 2, activation ='softmax') 
cnn = regression(cnn, optimizer ='adam', learning_rate = LR, 
      loss ='categorical_crossentropy', name ='targets') 
  
model = tflearn.DNN(cnn, tensorboard_dir ='log') 

In [55]:
#Oviya
#model.fit({'input': train_x}, {'targets': train_y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}),snapshot_step = 500, show_metric=True,run_id = MODEL_NAME )
#model.save(MODEL_NAME)

#--------------------using KERAS------------------------------#

#create our CNN Model
class MyModel(tf.keras.Model):
  def __init__(self):
    #define layers
    super(MyModel, self).__init__()
    self.layer1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.layer2 = tf.keras.layers.Dense(5, activation=tf.nn.relu)
  
  def call(self,inputs):
    x = self.layer1(inputs)
    return self.layer2(x)

#call model and send in inputs 
#input layer
#inputs = tf.keras.Input(shape=(3,), tensor=train_x)
model = MyModel()
model.compile(loss ='binary_crossentropy', metrics=['accuracy'])
model.fit(train_x,train_y, epochs = 150, steps_per_epoch = 5)
_,accuracy = model.evaluate(train_x,train_y)




ValueError: ignored